In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
from tqdm import tqdm

movies = pd.read_excel('movie15.xls', skiprows=4)
def genre(x):
    if '성인물(에로)' not in x:
        return x
movies['장르'] = movies.장르.apply(genre)
movies = movies[~movies['장르'].isnull()].reset_index(drop=True)
print(len(movies.장르.unique()))
movies.head()

885


,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,부활의 노래,The Song Of Resurrection,1990.0,한국,장편,드라마,개봉,이정국,새빛영화제작소
1,이매큘레이트,Immaculate,2024.0,미국,장편,공포(호러),개봉,마이클 모한,NaN
2,러브 라이즈 블리딩,Love Lies Bleeding,2023.0,미국,장편,"범죄,멜로/로맨스",개봉,로즈 글래스,NaN
3,짱,Top Dog,2018.0,한국,장편,액션,개봉,송재덕,"(주)영화사 사계절,펀펀한영화사"
4,싸움의 기술 2,The Art of Fighting 2,2020.0,한국,장편,액션,개봉,김희성,(주)영화사 사계절


In [2]:
movies.to_csv('movies_not_19.csv',index=False)

In [2]:
movies = pd.read_csv('movies_not_19.csv')
movies.head()

,영화명,영화명(영문),제작연도,제작국가,유형,장르,제작상태,감독,제작사
0,부활의 노래,The Song Of Resurrection,1990.0,한국,장편,드라마,개봉,이정국,새빛영화제작소
1,이매큘레이트,Immaculate,2024.0,미국,장편,공포(호러),개봉,마이클 모한,NaN
2,러브 라이즈 블리딩,Love Lies Bleeding,2023.0,미국,장편,"범죄,멜로/로맨스",개봉,로즈 글래스,NaN
3,짱,Top Dog,2018.0,한국,장편,액션,개봉,송재덕,"(주)영화사 사계절,펀펀한영화사"
4,싸움의 기술 2,The Art of Fighting 2,2020.0,한국,장편,액션,개봉,김희성,(주)영화사 사계절


In [3]:
movies = movies.영화명.tolist()
movies[:15]

['부활의 노래',
 '이매큘레이트',
 '러브 라이즈 블리딩',
 '짱',
 '싸움의 기술 2',
 '아이윌 송',
 '싸움의 기술',
 '깡패들',
 '전야',
 '심장박동 조작극',
 '비포 선셋',
 '사일런트 나잇',
 '명탐정 코난: 100만 달러의 펜타그램',
 '그리고 바통은 넘겨졌다',
 '더 납작 엎드릴게요']

In [4]:
url = 'https://namu.wiki/w/' # %EC%82%B4%EC%9D%B8%EC%9D%98%20%EC%B6%94%EC%96%B5
pd_movies = []
content = ''
cnt = 0
nums = list(range(10))

In [5]:
page = requests.get(url + '이매큘레이트')
html = BeautifulSoup(page.text, 'html.parser')

soup1 = html.select('#app div div div div') # 시놉시스
soup2 = html.select('#app div div div div .wiki-paragraph') # 테이블 형식 데이터 수집

st = 0
for s in soup1:
    if s.text != '':
        if '시놉시스[편집]' in s.text:
            content = s.text
            break
lst = content.split('[편집]')
for i, v in enumerate(lst):
    if v[-4:] == '시놉시스':
        synopsys = lst[i+1]
        break

In [6]:
synopsys

"순수하고 신실한 믿음을 가진 수녀 '세실리아'(시드니 스위니).어느 비밀스러운 수녀원의 초청을 받아 낯선 생활에 적응해 가던 중처녀의 몸으로 임신을 했다는 사실이 밝혀지고기적의 주인공으로 추앙 받기 시작하는데…순결한 수녀의 임신, 축복인가 저주인가!올여름, 충격적인 공포의 실체가 드러난다!영화진흥위원회 영화관입장권통합전산망5. 등장인물"

In [7]:
lst = [] # 영화에 대한 정보를 담는 리스트
for s in soup2[1:]: # 영화의 정보, 1은 총 데이터 묶음이므로 생략
    if s.text != '':
        if s.text == '장르':
            st = 1
        elif '개요' in s.text:
            st = 0
            break
        if st == 1:
            lst.append(s.text) # 영화 정보 담기
    if '개요' in s.text:
        break

In [8]:
lst.insert(0, '영화명')
lst.insert(1, 'movie')
lst.append('시놉시스')
lst.append(synopsys)
df = pd.DataFrame(lst)
df = pd.concat([df.iloc[::2].reset_index(drop=True), df.iloc[1::2].reset_index(drop=True)], axis=1)
df.columns = ['kind','content']
df.set_index(df.kind, inplace=True)
df = df.drop('kind', axis=1)
pd_movies.append(df)

In [9]:
pd_movies[0]

,content
kind,
영화명,movie
장르,"공포, 스릴러"
감독,마이클 모한
각본,앤드류 로벨
제작,"데이빗 버나드, 요나탄 다비노, 마이클 하임러, 테디 슈워츠먼, 시드니 스위니"
원작,출연
"시드니 스위니, 알바로 모르테 외",촬영
엘리샤 크리스티안,편집
크리스챤 마시니,미술


In [ ]:
pd_movies = []
url = 'https://namu.wiki/w/' 
nums = list(range(10))

for movie in tqdm(movies):
    content = ''
    synopsys = ""

    page = requests.get(url + movie)
    html = BeautifulSoup(page.text, 'html.parser')

    soup1 = html.select('#app div div div div') # 시놉시스
    soup2 = html.select('#app div div div div .wiki-paragraph') # 테이블 형식 데이터 수집

    for s in soup1:
        if s.text != '':
            if '시놉시스[편집]' in s.text:
                content = s.text
                break
    lst = content.split('[편집]')
    for i, v in enumerate(lst):
        if v[-4:] == '시놉시스':
            synopsys = lst[i+1]
            break

    st = 0
    lst = []
    for s in soup2[1:]: # 영화의 정보, 1은 총 데이터 묶음이므로 생략
        if s.text != '':
            if s.text == '장르':
                st = 1
            elif (st == 1) and ('개요' in s.text):
                st = 0
                break
            if st == 1:
                lst.append(s.text) # 영화 정보 담기
                if len(lst) > 70:
                    break

    if (lst != []) and (synopsys!=''):
        lst.insert(0, '영화명')
        lst.insert(1, movie)
        lst.append('시놉시스')
        lst.append(synopsys)
        df = pd.DataFrame(lst)
        df = pd.concat([df.iloc[::2].reset_index(drop=True), df.iloc[1::2].reset_index(drop=True)], axis=1)
        df.columns = ['kind','content']
        df.set_index(df.kind, inplace=True)
        df = df.drop('kind', axis=1)
        pd_movies.append(df)

In [14]:
pd_movies = []

url = 'https://namu.wiki/w/' 

for movie in tqdm(movies):
    content = ''
    synopsys = ""

    page = requests.get(url + movie)
    html = BeautifulSoup(page.text, 'html.parser')

    soup1 = html.select('#app div div div div') # 시놉시스

    for s in soup1:
        if s.text != '':
            if '시놉시스[편집]' in s.text:
                content = s.text
                break
    lst = content.split('[편집]')
    for i, v in enumerate(lst):
        if v[-4:] == '시놉시스':
            synopsys = lst[i+1]
            break
    
    if synopsys == '': # 영화를 못찾아서 시놉시스가 없는 경우는 해당 코드로
        content = ''
        synopsys = ""

        page = requests.get(url + movie+'(영화)')
        html = BeautifulSoup(page.text, 'html.parser')

        soup1 = html.select('#app div div div div')

        for s in soup1:
            if s.text != '':
                if '시놉시스[편집]' in s.text:
                    content = s.text
                    break
        lst = content.split('[편집]')
        for i, v in enumerate(lst):
            if v[-4:] == '시놉시스':
                synopsys = lst[i+1]
                break


    soup2 = html.select('#app div div div div .wiki-paragraph') # 테이블 형식 데이터 수집

    st = 0
    lst = []
    for s in soup2[1:]: # 영화의 정보, 1은 총 데이터 묶음이므로 생략
        if s.text != '':
            if s.text == '장르':
                st = 1
            elif (st == 1) and ('개요' in s.text):
                st = 0
                break
            if st == 1:
                lst.append(s.text) # 영화 정보 담기
                if len(lst) > 70:
                    break

    if (lst != []) and (synopsys!=''):
        lst.insert(0, '영화명')
        lst.insert(1, movie)
        lst.append('시놉시스')
        lst.append(synopsys)
        df = pd.DataFrame(lst)
        df = pd.concat([df.iloc[::2].reset_index(drop=True), df.iloc[1::2].reset_index(drop=True)], axis=1)
        df.columns = ['kind','content']
        df.set_index(df.kind, inplace=True)
        df = df.drop('kind', axis=1)
        pd_movies.append(df)

100%|██████████| 13306/13306 [1:58:21<00:00,  1.87it/s]  


In [15]:
len(pd_movies)

2663

In [18]:
pd_movies[110]

,content
kind,
영화명,30일
장르,어드벤처
출시,Google Play 2021년 8월 26일[4]App Store 2021년 12월...
비용,무료[5]
심의 등급,전체 이용가
링크,
시놉시스,"""나는 방금 사망한 이의 사망진단서를 받아들었다.나에게는 이 사람을 살릴 의무는 전..."


In [20]:
import pickle
with open('movies_none_lating.pkl', 'wb') as f:
    pickle.dump(pd_movies, f)

In [21]:
with open('movies_none_lating.pkl', 'rb') as f:
    d = pickle.load(f)

In [22]:
d[0]

,content
kind,
영화명,이매큘레이트
장르,"공포, 스릴러"
감독,마이클 모한
각본,앤드류 로벨
제작,"데이빗 버나드, 요나탄 다비노, 마이클 하임러, 테디 슈워츠먼, 시드니 스위니"
원작,출연
"시드니 스위니, 알바로 모르테 외",촬영
엘리샤 크리스티안,편집
크리스챤 마시니,미술
